In [57]:
import pandas as pd
import numpy as np
import os,glob
from matplotlib import pyplot as plt

In [58]:
a = pd.read_json('mass_json/first_mass_lda_at_16.json')

In [59]:
# a = j['anger'].copy()
a.shape

(7670, 12)

In [60]:
#write a function to flatten the features


In [61]:
a.columns

Index(['File_num', 'Title', 'Speaker', 'Year', 'Month', 'alpha', 'beta',
       'topic_lists', 'iter', 'time (sec)', 'log_probs', 'fifty_words'],
      dtype='object')

# does the replace command work in all elements?

In [62]:
def clean_words(feat):
    #len(feat)
    feat = list(feat)

    try:
        feat = list(filter(lambda a: a != '…', feat))
    except:
        pass
    #print(len(feat))

    try:
        feat = list(filter(lambda a: a != '…”', feat))
    except:
        pass
    try:
        feat = list(filter(lambda a: a != ']', feat))
    except:
        pass
    
    try:
        feat = list(filter(lambda a: a != '[', feat))
    except:
        pass
    #print(len(feat))
    # not necessary because it has already been cleaned
    feat = [x.replace('.','').replace('!','').replace('";','').replace('”','').replace('[','').replace(']','') for x in feat]
    feat = [x.replace('?','').replace('(','').replace(')','').replace('‘','') for x in feat]
    
    # len(feat)

    return feat

def combine_words(a,file_num):
    """
    parameters: 
        a (pandas dataframe): like in the format of LDA json files
        file_num (int):column to combine words for
    """
    feat = []
    for arr in a.loc[a.File_num == file_num].fifty_words:
        feat = np.append(feat,np.array(arr).flatten())
        
    # cleaning isn't necessary cuz we decided to do that in LDACGS
    new_feat = clean_words(feat)
    #new_feat = feat
    
    return new_feat 

In [63]:
# make new dataframe 
# b = a[['File_num', 'Title', 'Speaker', 'Year', 'Month']].drop_duplicates(subset='File_num',inplace=False)
print(a.shape)
b = a.drop_duplicates(subset='File_num',inplace=False)
b = b[['File_num', 'Title', 'Speaker', 'Year', 'Month','topic_lists']]
# we expect
print(b.shape)

(7670, 12)
(1534, 6)


In [64]:
words = []
for filenum in b.File_num.unique():
    words.append(combine_words(a,filenum))
b['words'] = words

In [65]:
b.columns

Index(['File_num', 'Title', 'Speaker', 'Year', 'Month', 'topic_lists',
       'words'],
      dtype='object')

# it might have `']'` as a feature but whatever

# weird words: fanfaresomething, chlorinated (from talk with ['mercy', 'patience', 'humility', 'service', 'dedication', 'fatherhood']) as topic_lists

In [66]:
# investigate results
rnd = np.random.choice(b.shape[0])
print(rnd)
print(b.iloc[rnd,-2])
print(len(b.iloc[rnd,-1]),'\n\n',b.iloc[rnd,-1])


895
['faith', 'sacrifice', 'obedience', 'temple work', 'covenants']
250 

 ['floods', 'abated', 'mortal', 'dispensation', 'repentance', 'temple', 'time', 'ordinances', 'blessed', 'president', 'thy', 'covenants', 'sons', 'covenant', 'temple', 'covenant', 'marriage', 'eternal', 'covenants', 'faithful', 'covenants', 'son', 'loss', 'grief', 'make', 'included', 'messengers', 'the', 'alma', 'prophets', 'gen', 'covenant', 'ye', 'establish', 'seed', 'noahs', 'lords', 'lord', 'sacrifice', 'day', 'lord', 'covenants', 'jesus', 'christ', 'sacred', 'noah', 'power', 'appointed', 'forces', 'sons', 'dc', 'included', 'messengers', 'faithful', 'abated', 'messengers', 'behold', 'included', 'gen', 'forces', 'covenants', 'make', 'covenant', 'keeping', 'lord', 'sacrifice', 'ye', 'struggle', 'covenant', 'christ', 'temple', 'time', 'door', 'president', 'motivational', 'beheld', 'dispensation', 'faithful', 'dc', 'preaching', 'president', 'lord', 'temple', 'were', 'sons', 'faithful', 'love', 'blessed', 'marriag

# create feature space based upon all the words

In [67]:
b.shape

(1534, 7)

In [68]:
all_key_words = set()
for i in range(b.shape[0]):
    all_key_words = all_key_words.union(set(b.iloc[i,-1]))
#print(len(all_key_words))

In [69]:
print(len(all_key_words))

13272


In [70]:
list_key_words = ['key_word:' + word for word in sorted(list(all_key_words))]
# list_key_words.remove('key_word:') # there are probably more like this

In [71]:
# a = 1000
# list_key_words[a:a + 50]

In [72]:
z = np.zeros((b.shape[0],len(all_key_words)))
Z = pd.DataFrame(z,columns = list_key_words)
#print(t.shape,Z.shape)
# b = pd.concat([b,Z],axis=1)
b.reset_index(inplace=True)
print('b.shape',b.shape)
#m for merged
m = b.merge(right=Z,how='inner',left_index=True,right_index=True)

b.shape (1534, 8)


In [73]:
print(Z.shape,b.shape,m.shape)

(1534, 13272) (1534, 8) (1534, 13280)


In [74]:
b.columns

Index(['index', 'File_num', 'Title', 'Speaker', 'Year', 'Month', 'topic_lists',
       'words'],
      dtype='object')

In [76]:
# m.head()

In [80]:
import time

In [82]:
m.shape[0]

1534

In [83]:
start = time.time()
# my algorithm for the one hot encoding will be to iterate through the rows, 
#an for the list of topics for that row to add one in the appropriate column
# this ran in just a few seconds
column_names = list(m.columns)
for i in range(m.shape[0]):
    individual_topics = m.iloc[i]['words']
    for word in individual_topics:
        m.iloc[i,column_names.index('key_word:' + word)] += 1
    if (i % 10) == 0:
        print(f'{np.round(i / m.shape[0] * 100,1)}% done')
print((time.time() - start)/60,' minutes to encode')

0.0% done
0.7% done
1.3% done
2.0% done
2.6% done
3.3% done
3.9% done
4.6% done
5.2% done
5.9% done
6.5% done
7.2% done
7.8% done
8.5% done
9.1% done
9.8% done
10.4% done
11.1% done
11.7% done
12.4% done
13.0% done
13.7% done
14.3% done
15.0% done
15.6% done
16.3% done
16.9% done
17.6% done
18.3% done
18.9% done
19.6% done
20.2% done
20.9% done
21.5% done
22.2% done
22.8% done
23.5% done
24.1% done
24.8% done
25.4% done
26.1% done
26.7% done
27.4% done
28.0% done
28.7% done
29.3% done
30.0% done
30.6% done
31.3% done
31.9% done
32.6% done
33.2% done
33.9% done
34.6% done
35.2% done
35.9% done
36.5% done
37.2% done
37.8% done
38.5% done
39.1% done
39.8% done
40.4% done
41.1% done
41.7% done
42.4% done
43.0% done
43.7% done
44.3% done
45.0% done
45.6% done
46.3% done
46.9% done
47.6% done
48.2% done
48.9% done
49.5% done
50.2% done
50.8% done
51.5% done
52.2% done
52.8% done
53.5% done
54.1% done
54.8% done
55.4% done
56.1% done
56.7% done
57.4% done
58.0% done
58.7% done
59.3% done
60.0

# it only took 2 minutes to encode 1534 * 250 words

In [89]:
# m.to_json('first_encoded_lda_dataset.json')

In [90]:
#merge with topics
path = 'other_files/'
f = 'merged_summary_topics.json'
t = pd.read_json(path + f)
# my data is in data2 not data
t['File'] = [x[5:] for x in t.File]
t['File_num'] = [int(x[:-4]) for x in t.File]
t['tag_count'] = [len(x) for x in t.topic_lists]

# MERGE ON File_num cuz it's unique to the talk 

In [93]:
print(m.shape,t.shape)

(1534, 13280) (3465, 309)


In [103]:
#tk for topic_keywords
tk = m.merge(t,how='inner',left_on='File_num',right_on='File_num')
# merge_cols = set(['File_num', 'Title', 'Speaker', 'Year', 'Month', 'topic_lists'])
# tk = m.merge(t,how='inner',left_on=merge_cols,right_on=merge_cols)

In [105]:
#there are duplicates columns, remove them 
for col in ['Title', 'Speaker', 'Year', 'Month', 'topic_lists']:
    tk.rename(columns={col + '_x':col},inplace=True)
    tk.drop(columns=col + '_y',inplace=True)

In [95]:
tk.shape

(1534, 13588)

In [98]:
m.columns

Index(['index', 'File_num', 'Title', 'Speaker', 'Year', 'Month', 'topic_lists',
       'words', 'key_word:', 'key_word:a',
       ...
       'key_word:zip', 'key_word:zippered', 'key_word:zone', 'key_word:zones',
       'key_word:zoram', 'key_word:zoramites', 'key_word:zrich',
       'key_word:zuleika', 'key_word:zwickau', 'key_word:zwingli'],
      dtype='object', length=13280)

In [107]:
# tk.to_json('third_encoded_LDA_with_topics.json')

In [106]:
tk.head()

,index,File_num,Title,Speaker,Year,Month,topic_lists,words,key_word:,key_word:a,...,womanhood,women,work,worldliness,worship,worthiness,young adults,young single adults,youth,tag_count
0,346,8292,The Voice of the Lord,Neil L. Andersen,2017,10,"[prophets, general conference]","[painting, heaven, eternal, lives, lord, elder...",0.0,0.0,...,0,0,0,0,0,0,0,0,0,2
1,351,8291,Love One Another as He Has Loved Us,José L. Alonso,2017,10,"[Jesus Christ, forgiveness, love, service]","[path, ted, forgive, family, love, love, faith...",0.0,0.0,...,0,0,0,0,0,0,0,0,0,4
2,356,8290,Seek Ye Out of the Best Books,Ian S. Ardern,2017,10,"[faith, patience, sabbath, scripture study, Bo...","[god, man, read, return, book, study, church, ...",0.0,0.0,...,0,0,0,0,0,0,0,0,0,7
3,361,8289,Essential Truths—Our Need to Act,Adilson de Paula Parrella,2017,10,"[Jesus Christ, prophets, God the Father, conve...","[brazil, young, engaged, wife, date, death, bo...",0.0,0.0,...,0,0,0,0,0,0,0,0,0,7
4,366,8288,Do We Trust Him? Hard Is Good,Stanley G. Ellis,2017,10,"[Jesus Christ, faith, sacrifice, adversity, Go...","[helaman, firmer, stronger, great, war, trust,...",0.0,0.0,...,0,0,0,0,0,0,0,0,0,7
